In [1]:
import pandas as pd

input_file = '..\data\Admission_Predict.csv'# 输入的数据文件
data = pd.read_csv(input_file, encoding='utf-8')  # 读取数据
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial_No.         400 non-null    int64  
 1   GRE_Score          400 non-null    int64  
 2   TOEFL_Score        400 non-null    int64  
 3   University_Rating  400 non-null    int64  
 4   SOP                400 non-null    float64
 5   LOR                400 non-null    float64
 6   CGPA               400 non-null    float64
 7   Research           400 non-null    int64  
 8   Chance_of_Admit    400 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 28.3 KB


In [2]:

# 异常值处理
# 计算均值和标准差
mean_value = data.mean()
std_dev = data.std()

# 定义异常值阈值
threshold = 3

# 筛选出异常值的索引
outliers = (data - mean_value).abs() > threshold * std_dev

# 删除异常值所在的行
data = data[~outliers]
data.dropna(inplace=True)

data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 399 entries, 0 to 399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial_No.         399 non-null    int64  
 1   GRE_Score          399 non-null    int64  
 2   TOEFL_Score        399 non-null    int64  
 3   University_Rating  399 non-null    int64  
 4   SOP                399 non-null    float64
 5   LOR                399 non-null    float64
 6   CGPA               399 non-null    float64
 7   Research           399 non-null    int64  
 8   Chance_of_Admit    399 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 31.2 KB


In [3]:

# 进行聚类分析

from sklearn.preprocessing import StandardScaler

# 假设 df 是你的 DataFrame，选择需要进行聚类的特征列
selected_columns = ['GRE_Score', 'TOEFL_Score', 'University_Rating', 'SOP', 'LOR', 'CGPA','Research']

# 提取所选列
data_for_clustering = data[selected_columns]

# 标准化数据（对 K-Means 敏感）
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_for_clustering)

from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score

# 存储不同 k 对应的 Calinski-Harabasz Index
calinski_scores = []

# 尝试不同的 k 值
for k in range(2, 20):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_for_clustering)
    labels = kmeans.labels_
    calinski_score = calinski_harabasz_score(data_for_clustering, labels)
    calinski_scores.append(calinski_score)

# 打印不同 k 对应的 Calinski-Harabasz Index
for k, score in zip(range(2, 20), calinski_scores):
    print(f'k={k}: Calinski-Harabasz Index = {score}')


k=2: Calinski-Harabasz Index = 718.8134798050091
k=3: Calinski-Harabasz Index = 725.6410461459409
k=4: Calinski-Harabasz Index = 853.2496933561921
k=5: Calinski-Harabasz Index = 775.5192608264063
k=6: Calinski-Harabasz Index = 757.4181173520806
k=7: Calinski-Harabasz Index = 709.1168833422685
k=8: Calinski-Harabasz Index = 660.3157712054929
k=9: Calinski-Harabasz Index = 597.926712629815
k=10: Calinski-Harabasz Index = 593.9214463642696
k=11: Calinski-Harabasz Index = 562.0633300978172
k=12: Calinski-Harabasz Index = 586.1718939926475
k=13: Calinski-Harabasz Index = 559.3899601118827
k=14: Calinski-Harabasz Index = 553.5945743817854
k=15: Calinski-Harabasz Index = 546.155477983729
k=16: Calinski-Harabasz Index = 521.7978708071455
k=17: Calinski-Harabasz Index = 544.1207204388796
k=18: Calinski-Harabasz Index = 536.2503864051654
k=19: Calinski-Harabasz Index = 529.2822716496066


In [4]:
# 创建 K-Means 模型
num_clusters = 4  # 设置聚类的簇数
kmeans = KMeans(n_clusters=num_clusters, random_state=42)

# 执行聚类
data['Cluster'] = kmeans.fit_predict(scaled_data)

# 打印每个簇的样本数量
print(data['Cluster'].value_counts())
# 按照聚类标签进行分组，并计算均值
cluster_means = data.groupby('Cluster').mean()

# 查看均值
print(cluster_means)
# 将聚类得到的各个类的各个特征的均值写入csv文件
cluster_means.to_csv('..\\temp\\K-means分析.csv')


Cluster
2    108
0    104
1    100
3     87
Name: count, dtype: int64
         Serial_No.   GRE_Score  TOEFL_Score  University_Rating       SOP  \
Cluster                                                                     
0        184.673077  329.961538   114.365385           4.451923  4.471154   
1        190.010000  312.300000   105.780000           2.900000  3.365000   
2        203.212963  318.759259   107.962963           2.990741  3.407407   
3        229.735632  304.034483   100.379310           1.816092  2.155172   

              LOR      CGPA  Research  Chance_of_Admit  
Cluster                                                 
0        4.370192  9.320865  0.923077         0.892500  
1        3.440000  8.480200  0.000000         0.680300  
2        3.384259  8.601759  1.000000         0.735741  
3        2.471264  7.889540  0.160920         0.564023  


In [5]:
# k-means++
# 尝试不同的 k 值
for k in range(2, 20):
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
    kmeans.fit(data_for_clustering)
    labels = kmeans.labels_
    calinski_score = calinski_harabasz_score(data_for_clustering, labels)
    calinski_scores.append(calinski_score)

# 打印不同 k 对应的 Calinski-Harabasz Index
for k, score in zip(range(2, 20), calinski_scores):
    print(f'k={k}: Calinski-Harabasz Index = {score}')


k=2: Calinski-Harabasz Index = 718.8134798050091
k=3: Calinski-Harabasz Index = 725.6410461459409
k=4: Calinski-Harabasz Index = 853.2496933561921
k=5: Calinski-Harabasz Index = 775.5192608264063
k=6: Calinski-Harabasz Index = 757.4181173520806
k=7: Calinski-Harabasz Index = 709.1168833422685
k=8: Calinski-Harabasz Index = 660.3157712054929
k=9: Calinski-Harabasz Index = 597.926712629815
k=10: Calinski-Harabasz Index = 593.9214463642696
k=11: Calinski-Harabasz Index = 562.0633300978172
k=12: Calinski-Harabasz Index = 586.1718939926475
k=13: Calinski-Harabasz Index = 559.3899601118827
k=14: Calinski-Harabasz Index = 553.5945743817854
k=15: Calinski-Harabasz Index = 546.155477983729
k=16: Calinski-Harabasz Index = 521.7978708071455
k=17: Calinski-Harabasz Index = 544.1207204388796
k=18: Calinski-Harabasz Index = 536.2503864051654
k=19: Calinski-Harabasz Index = 529.2822716496066


In [6]:
# 创建 K-Means 模型
num_clusters = 4  # 设置聚类的簇数
kmeans = KMeans(n_clusters=num_clusters, random_state=42)

# 执行聚类
data['Cluster'] = kmeans.fit_predict(scaled_data)

# 打印每个簇的样本数量
print(data['Cluster'].value_counts())


Cluster
2    108
0    104
1    100
3     87
Name: count, dtype: int64


In [7]:
# 按照聚类标签进行分组，并计算均值
cluster_means = data.groupby('Cluster').mean()

# 查看均值
print(cluster_means)
# 将聚类得到的各个类的各个特征的均值写入csv文件
cluster_means.to_csv('..\\temp\\K-means++分析.csv')


         Serial_No.   GRE_Score  TOEFL_Score  University_Rating       SOP  \
Cluster                                                                     
0        184.673077  329.961538   114.365385           4.451923  4.471154   
1        190.010000  312.300000   105.780000           2.900000  3.365000   
2        203.212963  318.759259   107.962963           2.990741  3.407407   
3        229.735632  304.034483   100.379310           1.816092  2.155172   

              LOR      CGPA  Research  Chance_of_Admit  
Cluster                                                 
0        4.370192  9.320865  0.923077         0.892500  
1        3.440000  8.480200  0.000000         0.680300  
2        3.384259  8.601759  1.000000         0.735741  
3        2.471264  7.889540  0.160920         0.564023  
